In [ ]:
# <-----------------> rellena con código 

In [ ]:
DOWNLOAD_DATASETS=False
if DOWNLOAD_DATASETS==True:
    !gammapy download datasets
import os
os.environ["GAMMAPY_DATA"] = "gammapy-datasets/1.0.1"

In [ ]:
from gammapy.catalog import SourceCatalog3FHL, SourceCatalogHGPS


In [ ]:
fermi_cat = SourceCatalog3FHL()
# <-----------------> rellenar con código 

# Información

In [ ]:
# Descripción de los catálogos
print(fermi_cat.description)
# <-----------------> rellenar con código 

In [ ]:
# Observa las información que hay guardada en cada catálogo.
# Una forma para verlo puede ser llamando al atributo table del objeto que representa un catálogo.
# Ejemplo: catalogo.table -> muestra la tabla con todos los objetos
# Ejemplo: catalogo.table[:5] -> muestra una tabla con las primeras 5 filas
# Ejemplo: lista_indices = [1,3,5,7] -> muestra una tabla con las filas 1,3,5 y 7

# XMATCH
Las posiciones en los catálogos se obtienen llamando al atributo 'positions'

In [ ]:
# Obtención de las posiciones.
## Crea dos variables llamadas fermi_pos y hess_pos que sean las coordenadas de los repectivos catálogos

# <-----------------> rellenar con código 
# <-----------------> rellenar con código 

In [ ]:
def naive_xmatch (x,y,min_distance=0.1*units.deg):
    """
    Args:
        x: lista con posiciones
        y: lista con posiciones
    Returns:
        xindex: lista de índices
        yindex: lista de índices
        dists: lista de distancias
        
    Devuelve los índeces de x e y que están a una distancia menor que 'min_distance' además de 
    la distancia de separación. La devolución se hace en orden, esto es, el primer índice de la lista
    xindex está a una distancia menor que min_distance del primero de yindex y, éstos, están a una distancia
    que corresponde al primer elemento de la lista dists
    """
    xindex = []
    yindex = []
    dists = []
    for i in range(len(x)):
        d = x[i].separation(y)
        amin = d.argmin()
        dmin = d[amin]
        if dmin < min_distance:
            xindex.append(i)
            yindex.append(amin)
            dists.append(dmin)
    return xindex,yindex,dists

In [ ]:
# Rellena la siguiente celda para obtener los índices que corresponden a los mismos objetos en cada catálogo
# <-----------------> rellenar con código 

In [ ]:
# Comprueba si el emparejamiento tienen sentido mirando las columnas de los catálogos
# <-----------------> rellenar con código 

# Estudio de objetos y ajustes

En este punto hay que ver entro los objetos que están en ambos catálogos y que se han unido en el 
xmatch anterior, de qué tipo son y buscar información sobre ellos. No hay que hacerlo para todosm pero, al menos, para tres objetos que sean de tipos distintos

Después hay que realizar distintos ajustes: uno para los datos de Fermi, otro para los de Hess y otro para ambos a la vez.

## Ejemplo de ajuste a los datos de Fermi

Supógase que se ha elegido el objeto 35 del catálogo de Fermi que corresponde con el objeto 53 del de H.E.S.S (esto se ha elegido de forma aleatoria, probablemente no coinciden)

### Construcción del dataset
```
fds = FluxPointsDataset(data=fermi_cat[35].flux_points,
                        name=fermi_cat[35].name)
hds = FluxPointsDataset(data=hess_cat[hess_idx_matches[53]].flux_points,
                        name=hess_cat[hess_idx_matches[53]].name)
joint_ds = Datasets([fds,hds])
```

### Gráfica con los datos
```
fig = plt.figure()
ax = fig.add_subplot(111)
plot_kwargs = {"ax":ax,"sed_type":"e2dnde"}
ax.yaxis.set_units(units.Unit("TeV cm-2 s-1"))
fds.data.plot(label=fds.name,**plot_kwargs)
hds.data.plot(label=hds.name,**plot_kwargs)
ebounds = [10,7e4]*units.GeV
ax.set_xlim(ebounds)
ax.legend()
```

![](graf_test_multiwave.png)

### Ajuste del conjunto de datos
```
spectral_model = gmodels.PowerLawSpectralModel(amplitude="1e-12 cm-2 s-1 TeV-1",
                                               reference="100 GeV")
sky_model = gmodels.SkyModel(spectral_model=spectral_model,
                             name=str(spectral_model).split()[0])
joint_ds.models = sky_model
fitter = Fit(backend="minuit")
fitter.run(datasets=joint_ds)
```

Mostrar los valores ajustados del modelo
```
sky_model.spectral_model.parameters.to_table()[["name","value","error"]]
```

### Grárfica con datos y ajuste
```
fig = plt.figure()
ax = fig.add_subplot(111)
plot_kwargs = {"ax":ax,"sed_type":"e2dnde"}
ax.yaxis.set_units(units.Unit("TeV cm-2 s-1"))
fds.data.plot(label=fds.name,**plot_kwargs)
hds.data.plot(label=hds.name,**plot_kwargs)
ebounds = [10,7e4]*units.GeV
sky_model.spectral_model.plot(ebounds,**plot_kwargs,
                              label=sky_model.name,
                             color="black")
sky_model.spectral_model.plot_error(ebounds,**plot_kwargs,
                                    color="gray")
ax.set_xlim(ebounds)
ax.legend()
```

![](graf_test_multiwave_fit.png)

### Cómo valorar el ajuste

Las formas de valorar la bondad de un ajuste suelen basarse en los residuos.
Los residuos se calculan usando los valores reales y los predichos.

Si joint_ds es un dataset que contiene (datos de Fermi, datos de HESS), entonces:

```
y_fermi_true = joint_ds[0].data.to_table("dnde")["dnde"].quantity
y_hess_true = joint_ds[1].data.to_table("dnde")["dnde"].quantity
y_fermi_pred = joint_ds[0].flux_pred()
y_hess_pred = joint_ds[1].flux_pred()

res_sum = (abs(y_fermi_pred-y_fermi_true)).sum() + (abs(y_hess_pred-y_hess_true)).sum()
```

In [ ]:
# <-----------------> rellenar con código, tantas celdas como se necesiten para realizar el análisis